# Formula 1

In this example we will consider a path-following problem in the presence of an obstacle. In this example we will se how 

1. to add hard generic stage and terminal constraints to the MPC.
2. to add soft generic stage and terminal constraints to the MPC.

<img src="../images/f1_example.png" alt="drawing" width="800"/>


## Introduction
We consider the problem of a racing F1 car along a ellipsoidal track. At some point of the track an obstacle is present due to an accident. For the car model we use a simple bicycle model:

<img src="../images/bike.png" alt="drawing" width="600"/>


the model equation look like:

$$
\begin{align}
\dot{p}_X &= v \cos (\psi + \beta), \\
\dot{p}_Y &= v \sin (\psi + \beta), \\
\dot{v} &= a, \\
\dot{\psi} &= \frac{v}{l_r} \sin({\beta}), \\
\beta &= \tan^{-1}\Big(\frac{l_r}{l_f + l_r} \tan({\delta})\Big),
\end{align}
$$

the path to follow is an ellipse with the following parametric equation:

$$
F(\theta) = [\tau_X(\theta), \tau_Y(\theta)] = [30 - 14  \cos(\theta), \,\  30 - 16  \sin(\theta)] 
$$

the objective function looks like

$$
L = \sum_{i=1}^N \left [(p_{X,i}- \tau_X(\theta_i)) + (p_{Y,i}- \tau_Y(\theta_i)) \right] + (p_{X,{N+1}}- \tau_X(\theta_{N+1})) + (p_{Y,{N+1}}- \tau_Y(\theta_{N+1}))
$$

where $\theta$ is the path variable.

The obstacle is circular with radious $r=2$ centered in $(obs_{X}, obs_{Y}) = (30, 15)$ will be modelled by adding the following nonlinear constraints

$$
c(p_X,p_Y) = (p_X - obs_X)^2 + (p_Y - obs_Y)^2 \geq r^2
$$

## Simulation
we start importing the necessary modules

In [1]:
# Add HILO-MPC to path. NOT NECESSARY if it was installed via pip.
import sys
from hilo_mpc import Model, NMPC
import casadi as ca
import pandas as pd
import numpy as np

# Necessary for plots
import yaml
from bokeh.layouts import column, layout, row
from bokeh.models import ColumnDataSource, Slider, ImageURL, Band
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()

Loading BokehJS ...

### Model

In [2]:
model = Model(plot_backend='bokeh')

states = model.set_dynamical_states(['px', 'py', 'v', 'phi'])
inputs = model.set_inputs(['a', 'delta'])

# Unwrap states
px = states[0]
py = states[1]
v = states[2]
phi = states[3]

# Unwrap states
a = inputs[0]
delta = inputs[1]

# Parameters
lr = 1.4  # [m]
lf = 1.8  # [m]
beta = ca.arctan(lr / (lr + lf) * ca.tan(delta))

# ODE
dpx = v * ca.cos(phi + beta)
dpy = v * ca.sin(phi + beta)
dv = a
dphi = v / lr * ca.sin(beta)


model.set_dynamical_equations([dpx, dpy, dv, dphi])

# Initial conditions
x0 = [15, 30, 0, 0]
u0 = [0., 0.]

# Create model and run simulation
dt = 0.1
model.setup(dt=dt)
model.set_initial_conditions(x0=x0)

### Obstacle
We define a round obstacle with radius of 2 meters occupying almost the entire road cross-section. 

In [3]:
# Obstacle position and geometry
obs_x = 30
obs_y = 15 
obs_rad = 2 #[m]

### Setup the MPC

In [4]:
# Setup NMPC
nmpc = NMPC(model)


theta = nmpc.create_path_variable(u_pf_lb=0.2, u_pf_ub=1)

# You can also put the reference on the virtual input
# theta = nmpc.create_path_variable(u_pf_lb=0.1, u_pf_ub=1, u_pf_ref=None, u_pf_weight=10)

path_x = 30 - 14 * ca.cos(theta)
path_y = 30 - 16 * ca.sin(theta)

nmpc.horizon = 30
nmpc.quad_stage_cost.add_states(names=['px', 'py'], ref=ca.vertcat(path_x, path_y), weights=[1, 1], 
                                path_following=True)
nmpc.quad_stage_cost.add_inputs(names=['a','delta'], weights=[1,1])

nmpc.quad_terminal_cost.add_states(names=['px', 'py'], ref=ca.vertcat(path_x, path_y), weights=[1, 1], 
                                   path_following=True)

nmpc.set_box_constraints(x_lb=[-100, -100, -10, -100], x_ub=[100, 100, 10, 100], u_lb=[-1, -1], u_ub=[1, 1])

C:\Users\Bruno\Documents\python_packages\examples\lib\site-packages\hilo_mpc\modules\base.py:2174: UserWarning: Plots are disabled, since no backend was selected.
  warnings.warn("Plots are disabled, since no backend was selected.")


### Add nonlinear stage and terminal constraints 

In [5]:
nmpc.stage_constraint.constraint = (px - obs_x) ** 2 + (py - obs_y) ** 2
nmpc.stage_constraint.ub = ca.inf
nmpc.stage_constraint.lb = obs_rad ** 2



nmpc.terminal_constraint.constraint = (px - obs_x) ** 2 + (py - obs_y) ** 2
nmpc.terminal_constraint.ub = ca.inf
nmpc.terminal_constraint.lb = obs_rad ** 2


nmpc.set_initial_guess(x_guess=x0, u_guess=[0, 0])
nmpc.setup(options={'print_level':1})

For plotting we need to create

1. a line representing the path
2. a dataframe were to store the mpc predictions at every iteration

In [6]:
# Create function to plot the path
pp = ca.SX.sym('pp')
path_x = 30 - (14*pp) * ca.cos(theta)
path_y = 30 - (16*pp) * ca.sin(theta)

path_fun = ca.Function('path',[theta, pp],[path_x,path_y])

x_path = []
y_path = []
for t in range(800):
    x_p, y_p = path_fun(t / 100, 1)
    x_path.append(float(x_p))
    y_path.append(float(y_p))
    

In [7]:
# Define some dataframe where the results will be stored. This is necessary only for the animation beo
n_steps = 500
solution = model.solution

data = {}
data_animation = {}
for k in range(n_steps):
    for state in model.dynamical_state_names:
        data[(f'{k}',state)] = np.zeros((nmpc.horizon+1,1)).squeeze()
        
    data_animation[(f'{k}','url')] = ["../images/f1.png"]
    data_animation[(f'{k}','f1_x')] = [0]
    data_animation[(f'{k}','f1_y')] = [0]
    data_animation[(f'{k}','angle')] = [0]
    
df = pd.DataFrame(data=data)
df_animation = pd.DataFrame(data=data_animation)

### Simulation
The remaining code runs the simulation and stores the data for visualization

In [8]:
for i in range(n_steps):
    u = nmpc.optimize(x0)
    model.simulate(u=u)
    x0 = solution['x:f']
    x_pred, _, _ = nmpc.return_prediction()
    
    for k, state in enumerate(model.dynamical_state_names):
        df.loc[:,(f'{i}',state)] = x_pred[k,:]
    
    df_animation.loc[:,(f'{i}','f1_x')] = x_pred[0,0]
    df_animation.loc[:,(f'{i}','f1_y')] = x_pred[1,0]
    df_animation.loc[:,(f'{i}','angle')] = x_pred[3,0]#*180/ca.pi


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Ups... NMPC had some problems. The ipopt error message is: restoration_failed. Please refer to ipopt documentation. For more info pass ipopt.print_level:5 to the solver_options in the NMPC.setup()
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.

NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC found an optimal solution.
NMPC fou

NMPC found an optimal solution.
NMPC found an optimal solution.


## Results
The next code is only for the animated visualization of the results, hence not necessary in general.

In [9]:
def bkapp(doc):
    source = ColumnDataSource(data=df['0'])
    source_anim = ColumnDataSource(data=df_animation['0'])

    plot = figure(y_range=(10, 50), x_range = (10,50),
                  y_axis_label='y',x_axis_label='x',
                  title="F1 racing with obstacle.")

    plot.line(x=x_path, y=y_path,alpha=0.8, color='grey', line_width = 30)
#     plot.line(x=x_path_inner, y=y_path_inner, alpha=1, color='black', line_width = 1)
#     plot.line(x=x_path_outer, y=y_path_outer, alpha=1, color='black', line_width = 1)
    plot.line(x=x_path, y=y_path,line_dash='dashed', color='white')
    
    plot.line(x='px', y='py', source=source, color='red', line_width=3, legend_label ='prediction')

    
    image1 = ImageURL(url="url", x='f1_x', y='f1_y', angle='angle', w=3, h=3, anchor="center")
    
    plot.add_glyph(source_anim, image1)

    r = plot.circle(x=obs_x, y = obs_y)
    glyph = r.glyph
    glyph.radius = obs_rad/2
    glyph.fill_alpha = 0.9
    glyph.fill_color = 'black'
    glyph.line_color = "firebrick"
    glyph.line_width = 2
    
    def callback(attr, old, new):
        if new == 0:
            data = df['0']
            data_anim = df_animation['0']
        else:
            data = df['{}'.format(new)]
            data_anim = df_animation['{}'.format(new)]
            
        source.data = ColumnDataSource.from_df(data)
        source_anim.data = ColumnDataSource.from_df(data_anim)
        
    slider = Slider(start=0, end=n_steps-1, value=0, step=1, title="Sampling time")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#fafafa"
                outline_line_color: black
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: gray
    """, Loader=yaml.FullLoader))

In [10]:
show(bkapp,notebook_url="http://localhost:8888" )

## Soft constraints
We test hat happens if the constraints are soft

In [11]:
model.reset_solution()
model.setup(dt = dt)
# Initial conditions
x0 = [15, 30, 0, 0]
model.set_initial_conditions(x0=x0)

In [12]:
# Setup NMPC
nmpc = NMPC(model)


theta = nmpc.create_path_variable(u_pf_lb=0.2, u_pf_ub=1)
path_x = 30 - 14 * ca.cos(theta)
path_y = 30 - 16 * ca.sin(theta)

nmpc.horizon = 30
nmpc.quad_stage_cost.add_states(names=['px', 'py'], ref=ca.vertcat(path_x, path_y), weights=[1, 1], path_following=True)
# nmpc.quad_stage_cost.add_inputs(names=['a', 'delta'], weights=[1, 5])
nmpc.quad_terminal_cost.add_states(names=['px', 'py'], ref=ca.vertcat(path_x, path_y), weights=[1, 1], path_following=True)
nmpc.set_box_constraints(x_lb=[-100, -100, -10, -100], x_ub=[100, 100, 10, 100], u_lb=[-1, -1], u_ub=[1, 1])

C:\Users\Bruno\Documents\python_packages\examples\lib\site-packages\hilo_mpc\modules\base.py:2174: UserWarning: Plots are disabled, since no backend was selected.
  warnings.warn("Plots are disabled, since no backend was selected.")


In [13]:
nmpc.stage_constraint.constraint = (px - obs_x) ** 2 + (py - obs_y) ** 2
nmpc.stage_constraint.ub = ca.inf
nmpc.stage_constraint.lb = obs_rad ** 2
nmpc.stage_constraint.is_soft =True
nmpc.stage_constraint.max_violation = 0.5

nmpc.terminal_constraint.constraint = (px - obs_x) ** 2 + (py - obs_y) ** 2
nmpc.terminal_constraint.ub = ca.inf
nmpc.terminal_constraint.lb = obs_rad ** 2
nmpc.terminal_constraint.is_soft = True
nmpc.terminal_constraint.max_violation = 0.5

nmpc.set_initial_guess(x_guess=x0, u_guess=[0, 0])
nmpc.setup(options={'print_level':0})


In [14]:
# Define some dataframe where the results will be stored. This is necessary only for the animation beo
n_steps = 150
solution = model.solution

data = {}
data_animation = {}
for k in range(n_steps):
    for state in model.dynamical_state_names:
        data[(f'{k}',state)] = np.zeros((nmpc.horizon+1,1)).squeeze()
        
    data_animation[(f'{k}','url')] = ["../images/f1.png"]
    data_animation[(f'{k}','f1_x')] = [0]
    data_animation[(f'{k}','f1_y')] = [0]
    data_animation[(f'{k}','angle')] = [0]
    data_animation[(f'{k}','eps_value')] = [0]
        
df_2 = pd.DataFrame(data=data)
df_animation_2 = pd.DataFrame(data=data_animation)

In [15]:
for i in range(n_steps):
    u = nmpc.optimize(x0)
    model.simulate(u=u)
    x0 = solution['x:f']
    x_pred, _, _ = nmpc.return_prediction()
    
    for k, state in enumerate(model.dynamical_state_names):
        df_2.loc[:,(f'{i}',state)] = x_pred[k,:]
    
    df_animation_2.loc[:,(f'{i}','f1_x')] = x_pred[0,0]
    df_animation_2.loc[:,(f'{i}','f1_y')] = x_pred[1,0]
    df_animation_2.loc[:,(f'{i}','angle')] = x_pred[3,0]#*180/ca.pi
    df_animation_2.loc[:,(f'{i}','eps_value')] = float(nmpc.stage_constraint.e_soft_value)
    df_animation_2.loc[:,(f'{i}','violation')] = 'e_stage_const'

C:\Users\Bruno\Documents\python_packages\examples\lib\site-packages\pandas\core\indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


In [16]:
def bkapp(doc):
    source = ColumnDataSource(data=df_2['0'])
    source_anim = ColumnDataSource(data=df_animation_2['0'])

    plot = figure(y_range=(10, 50), x_range = (10,50),
                  y_axis_label='y',x_axis_label='x',
                  title="F1 racing with obstacle.")

    plot.line(x=x_path, y=y_path,alpha=0.8, color='grey', line_width = 30)
#     plot.line(x=x_path_inner, y=y_path_inner, alpha=1, color='black', line_width = 1)
#     plot.line(x=x_path_outer, y=y_path_outer, alpha=1, color='black', line_width = 1)
    plot.line(x=x_path, y=y_path,line_dash='dashed', color='white')
    
    plot.line(x='px', y='py', source=source, color='red', line_width=3, legend_label ='prediction')

    # Vertical bar
    plot2 = figure(x_range=['e_stage_const'], width=150, y_range =(0,1e-5))
    plot2.vbar(x='violation', top='eps_value', width=0.9, source=source_anim)
    plot2.xgrid.grid_line_color = None
    plot2.y_range.start = 0
    
    # Car pic
    image1 = ImageURL(url="url", x='f1_x', y='f1_y', angle='angle', w=3, h=3, anchor="center")
    
    plot.add_glyph(source_anim, image1)

    r = plot.circle(x=obs_x, y = obs_y)
    glyph = r.glyph
    glyph.radius = obs_rad/2
    glyph.fill_alpha = 0.9
    glyph.fill_color = 'black'
    glyph.line_color = "firebrick"
    glyph.line_width = 2
    
    def callback(attr, old, new):
        if new == 0:
            data = df_2['0']
            data_anim = df_animation_2['0']
        else:
            data = df_2['{}'.format(new)]
            data_anim = df_animation_2['{}'.format(new)]
            
        source.data = ColumnDataSource.from_df(data)
        source_anim.data = ColumnDataSource.from_df(data_anim)
        
    slider = Slider(start=0, end=n_steps-1, value=0, step=1, title="Sampling time")
    slider.on_change('value', callback)

    doc.add_root(layout(slider, row(plot,plot2)))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#fafafa"
                outline_line_color: black
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: gray
    """, Loader=yaml.FullLoader))

In [17]:
show(bkapp,notebook_url="http://localhost:8888" )